In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


### Data preprocessing

In [2]:
train=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv') 

In [3]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [10]:
train.dtypes

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
                  ...   
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
SalePrice          int64
Length: 81, dtype: object

### Missing data

In [8]:
missing_data=train.isnull().sum().sort_values(ascending=False)
missing_data.head(10)

PoolQC         1453
MiscFeature    1406
Alley          1369
Fence          1179
MasVnrType      872
FireplaceQu     690
LotFrontage     259
GarageYrBlt      81
GarageCond       81
GarageType       81
dtype: int64

In [10]:
train=train.drop(missing_data.loc[missing_data>100].index, axis=1)
test=test.drop(missing_data.loc[missing_data>100].index, axis=1)

In [11]:
test.isnull().sum().sort_values(ascending=False).head(10)

GarageYrBlt     78
GarageFinish    78
GarageQual      78
GarageCond      78
GarageType      76
BsmtCond        45
BsmtExposure    44
BsmtQual        44
BsmtFinType2    42
BsmtFinType1    42
dtype: int64

In [12]:
train.isnull().sum().sort_values(ascending=False).head(10)

GarageType      81
GarageCond      81
GarageYrBlt     81
GarageFinish    81
GarageQual      81
BsmtFinType2    38
BsmtExposure    38
BsmtFinType1    37
BsmtCond        37
BsmtQual        37
dtype: int64

### Categorical

In [15]:
col_num=train.select_dtypes(include=np.number).columns
col_obj=train.select_dtypes(include='object').columns
print('Numerique', len(col_num))
print('Object', len(col_obj))

Numerique 37
Object 37


In [16]:
train[col_obj].nunique().sort_values(ascending=False)

Neighborhood     25
Exterior2nd      16
Exterior1st      15
SaleType          9
Condition1        9
Condition2        8
HouseStyle        8
RoofMatl          8
Functional        7
Foundation        6
GarageType        6
Heating           6
BsmtFinType2      6
BsmtFinType1      6
SaleCondition     6
RoofStyle         6
BldgType          5
GarageCond        5
GarageQual        5
Electrical        5
HeatingQC         5
LotConfig         5
MSZoning          5
ExterCond         5
BsmtExposure      4
BsmtCond          4
ExterQual         4
BsmtQual          4
KitchenQual       4
LandContour       4
LotShape          4
LandSlope         3
GarageFinish      3
PavedDrive        3
Utilities         2
Street            2
CentralAir        2
dtype: int64

In [20]:
train['Neighborhood'].unique()

array(['CollgCr', 'Veenker', 'Crawfor', 'NoRidge', 'Mitchel', 'Somerst',
       'NWAmes', 'OldTown', 'BrkSide', 'Sawyer', 'NridgHt', 'NAmes',
       'SawyerW', 'IDOTRR', 'MeadowV', 'Edwards', 'Timber', 'Gilbert',
       'StoneBr', 'ClearCr', 'NPkVill', 'Blmngtn', 'BrDale', 'SWISU',
       'Blueste'], dtype=object)

In [22]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline 

In [23]:
for col in col_obj:
    label=LabelEncoder()
    train[col]=label.fit_transform(train[col])

### Models

In [45]:
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold

In [ ]:
y_train=train['SalePrice']
X_train=train.drop(['Id', 'SalePrice'], axis=1)
X_train, X_val, y_train, y_val=train_test_split(X_train, y_train,test_size=0.2)

### LinearRegression

In [48]:
pipe=Pipeline([('imp', KNNImputer()), ('lr', LinearRegression())])
pipe.fit(X_train, y_train)

Pipeline(steps=[('imp', KNNImputer()), ('lr', LinearRegression())])

In [50]:
print(mse(np.log(pipe.predict(X_train)), np.log( y_train)))
print(mse(np.log(pipe.predict(X_val)), np.log( y_val)))

0.025039221214869914
0.03361274882150708


### Ensemble models
### Random forest

In [51]:
pipe_rdf=Pipeline([('imp', KNNImputer()), ('rdf', RandomForestRegressor(n_estimators=10))])
pipe_rdf.fit(X_train, y_train)

Pipeline(steps=[('imp', KNNImputer()),
                ('rdf', RandomForestRegressor(n_estimators=10))])

In [52]:
print(mse(np.log(pipe_rdf.predict(X_train)), np.log( y_train)))
print(mse(np.log(pipe_rdf.predict(X_val)), np.log( y_val)))

0.005088900310144752
0.022521763911746418


### Adaptive Boosting

In [67]:
pipe_adb=Pipeline([('imp', KNNImputer()), ('adb', AdaBoostRegressor(n_estimators=50))])
pipe_adb.fit(X_train, y_train)

Pipeline(steps=[('imp', KNNImputer()), ('adb', AdaBoostRegressor())])

In [68]:
print(mse(np.log(pipe_adb.predict(X_train)), np.log( y_train)))
print(mse(np.log(pipe_adb.predict(X_val)), np.log( y_val)))

0.03403745084603896
0.042615097474963906


### Gradient Boosting

In [71]:
pipe_gb=Pipeline([('imp', KNNImputer()), ('rdf',GradientBoostingRegressor())])
pipe_gb.fit(X_train, y_train)

Pipeline(steps=[('imp', KNNImputer()), ('rdf', GradientBoostingRegressor())])

In [72]:
print(mse(np.log(pipe_gb.predict(X_train)), np.log( y_train)))
print(mse(np.log(pipe_gb.predict(X_val)), np.log( y_val)))

0.007861163049178777
0.017774268168064995
